In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
test = pd.read_csv("./loansTest.csv")
train = pd.read_csv("./loansTrain.csv")

In [ ]:
# sampleSubmission = pd.read_csv("../input/machine-learning-course-project-cs-412/sampleSubmission.csv")

In [ ]:
# sampleSubmission.head()

In [ ]:
train

In [ ]:
test.dtypes

In [ ]:
test.columns[1:]

In [ ]:
# extract feature columns
feature_cols = list(train.columns[:-1])
# extract target column 'loan_status'
target_col = train.columns[-1]

# separate the data into feature data and target data
X_all = train[feature_cols]
y_all = train[target_col]

# extract feature columns
feature_cols = list(test.columns[1:])
# extract target column 'loan_status'
target_col = test.columns[0]

# separate the data into feature data and target data
X_test = test[feature_cols]
id_test = test[target_col]

X_all = pd.concat([X_all, X_test], ignore_index=True, sort=False)

In [ ]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['36 months', '60 months'], [0, 1])
            col_data = col_data.replace([' 36 months', ' 60 months'], [0, 1])


        print(col)
        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#Dropping columns with missing value rate higher than threshold
X_all = X_all[X_all.columns[X_all.isnull().mean() < 0.7]]


In [ ]:
# all missing values unemployed
X_all['emp_title'].fillna("unemployed",inplace=True)

# all titles lower case
X_all['emp_title'].str.lower()



# replace null with zeros
for col in X_all.columns:
    # Replacing the missing values with the maximum occurred value 
    X_all[col].fillna(X_all[col].value_counts().idxmax(), inplace=True)
    #X_all[col].fillna(0,inplace=True) 
    


In [ ]:
frequent_titles = pd.Series(' '.join(X_all['emp_title']).lower().split()).value_counts()[:150]
arr = frequent_titles.index.to_numpy()

to_be_ignored = ['the', 'of', 'and', '&', 'city', '-', '/','sr.', 'sr']

for ingnore in to_be_ignored:
    arr = np.delete(arr, np.argwhere(arr == ingnore))
    
#import re
#arr = map(lambda x:re.compile(x), arr )
    
#arr = '|'.join(arr)
#arr = set(arr)
#iscomedy = arr.issubset
#temp = X_all['emp_title'].str.contains(arr, regex=True)
for r in arr:
    X_all.replace(to_replace=r, value=r, regex=True )

In [93]:
new = temp.values
unique, counts = np.unique(new, return_counts=True)
dict(zip(unique, counts))

{False: 127845, True: 1175760}

In [58]:
X_all[X_all.str.contains('|'.join(frequent_titles))]

AttributeError: 'DataFrame' object has no attribute 'str'

In [13]:
# binning
#revol_util
#tot_coll_amt

In [14]:
# feature removel
X_all.drop(['zip_code', 'dti', 'avg_cur_bal','application_type','chargeoff_within_12_mths','acc_now_delinq', 'emp_title', 'delinq_amnt','percent_bc_gt_75','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','tax_liens'], axis=1, inplace=True)
# feature removel
# X_test.drop(['zip_code', 'dti', 'avg_cur_bal', 'emp_title'], axis=1, inplace=True)

In [15]:
# X_all = pd.get_dummies(X_all)
X_all = preprocess_features(X_all)
X, X_test = X_all.iloc[:1000000,:], X_all.iloc[1000000:,:]
X_train, X_val, y_train, y_val = train_test_split(X, y_all, train_size=0.8)

loan_amnt
term
grade
emp_length
home_ownership
annual_inc
verification_status
purpose
addr_state
delinq_2yrs
earliest_cr_line
inq_last_6mths
open_acc
pub_rec
revol_bal
revol_util
collections_12_mths_ex_med
tot_coll_amt
tot_cur_bal
total_rev_hi_lim
acc_open_past_24mths
bc_open_to_buy
bc_util
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_inq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_op_past_12m
pct_tl_nvr_dlq
pub_rec_bankruptcies
tot_hi_cred_lim
total_bal_ex_mort
total_il_high_credit_limit


In [16]:
from time import time
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds",format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label=1)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

In [17]:
clf = DecisionTreeClassifier()

In [18]:
train_predict(clf, X_train, y_train, X_val, y_val)

Training a DecisionTreeClassifier using a training set size of 800000. . .
Trained model in {:.4f} seconds 270.3956549167633
Made predictions in 9.8727 seconds.
F1 score for training set: 1.0000.
Made predictions in 1.1323 seconds.
F1 score for test set: 0.8239.


In [19]:
y_pred = clf.predict(X_test)

#predict_labels(clf, X_test, y_test)

In [20]:
len(y_pred)

303605

In [21]:
loan = pd.DataFrame({'loan_status': y_pred[:]})

In [22]:
loan

,loan_status
0,1
1,1
2,1
3,1
4,1
...,...
303600,1
303601,0
303602,0
303603,1


In [23]:
result = pd.concat([id_test,loan], axis=1, sort=False)

In [24]:
result

,ID,loan_status
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
303600,303600,1
303601,303601,0
303602,303602,0
303603,303603,1


In [26]:
result.to_csv('result.csv', index=False)

TypeError: sequence item 2: expected str instance, float found